In [1]:
# setup
from IPython.core.display import display,HTML
display(HTML('<style>.prompt{width: 0px; min-width: 0px; visibility: collapse}</style>'))
display(HTML(open('../rise.css').read()))

# imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(style="whitegrid", font_scale=1.5, rc={'figure.figsize':(12, 6)})


# CMPS 2200
# Introduction to Algorithms

## Shortest Path


Today's agenda:

- ...

## Generalizing DFS and BFS search

We've now seen two types of search that are conceptually very similar.


`dfs_stack`:

- `node = frontier.pop()`


`bfs_serial`:

- `node = frontier.popleft()`

Can we generalize these?

In [2]:
from collections import deque

def generic_search(graph, source, get_next_node_fn):
    def generic_search_helper(visited, frontier):
        if len(frontier) == 0:
            return visited
        else:
            ## pick a node
            node = get_next_node_fn(frontier)
            print('visiting', node)
            visited.add(node)
            frontier.extend(filter(lambda n: n not in visited, graph[node]))
            return generic_search_helper(visited, frontier)
        
    frontier = deque()
    frontier.append(source)
    visited = set()
    return generic_search_helper(visited, frontier)

def bfs_fn(frontier):
    return frontier.pop()

def dfs_fn(frontier):
    return frontier.popleft()

graph = {
            'A': {'B', 'C'},
            'B': {'A', 'D', 'E'},
            'C': {'A', 'F', 'G'},
            'D': {'B'},
            'E': {'B', 'H'},
            'F': {'C'},
            'G': {'C'},
            'H': {'E'}
        }

generic_search(graph, 'A', bfs_fn)

visiting A
visiting B
visiting E
visiting H
visiting D
visiting C
visiting F
visiting G


{'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'}

In [3]:
generic_search(graph, 'A', dfs_fn)

visiting A
visiting C
visiting B
visiting G
visiting F
visiting D
visiting E
visiting H


{'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'}

## Priority First Search

We can view the `get_next_node_fn` as a way to pick the **highest priority** node to visit at each iteration.

E.g., consider a Web crawler that prioritizes which pages to visit first
- more intereseting pages
- pages that update more frequently


We'll see several algorithms that are instances of priority first search. 

## Weighted graphs

Up to now we have focused on unweighted graphs. 

For many problems, we need to associate real-valued **weights** to each edge.

E.g., consider a graph where nodes are cities and edges represent the distance between them. 

<img src="figures/weighted.png" width=70%/>

The **weight of a path** in the graph is the sum of the weights of the edges along that path.

The **shortest weighted path** (or just **shortest path**) between **s** and **e** is the one with minimal weight.

**What is the shortest path from s to e**?

We saw that we can use BFS to get the distance from the source to each node.

Can we use BFS to solve the shortest path problem for weighted graphs?

<img src="figures/bfs_fail.png" width=60%/>

BFS will:
- visit b
- visit a
- but, will not visit path from a to b, since it doesn't visit a node more than once

Thus, BFS will not discover that the shortest path from $s$ to $b$ is $s \rightarrow a \rightarrow b$.

Consider another example:

<img src="figures/inf.png" width=60%/>

What is the shortest path from $s$ to $e$?

> Infinite loop in the cycle $s \rightarrow a \rightarrow b \rightarrow a$, so shortes path is $-\infty$

### SSSP: Single-Source Shortest Path

Given a weighted graph $G=(V,E,w)$ and a source vertex $s$, the single-source shortest path (SSSP) problem is to find a shortest weighted path from $s$ to every other vertex in $V$.

What would be a brute-force solution to this problem?

Is there anything we could reuse to improve the brute-force solution?


Consider this figure:

<img src="figures/subpaths.png" width="40%"/>

Suppose that an oracle has told us the shortest paths from $s$ to all vertices except for the vertex $v$, shown in red squares. How can we find the shortest path to $v$?



$\min(\delta_G(s,a)+3, \delta_G(s,b)+6, \delta_G(s,c)+5 )$

### sub-paths property
> any sub-path of a shortest path is itself a shortest path. 

The sub-paths property makes it possible to construct shortest paths from smaller shortest paths. 

> If a shortest path from Pittsburgh to San Francisco goes through Chicago, then that shortest path includes the shortest path from Pittsburgh to Chicago, and from Chicago to San Francisco.

- define path weights and distances
- problem types: 
  - single pair
  - single source
  - all pairs

- dijkstra
- analysis